In [1]:
#From anaconda prompt run this command for Gmsh installation: pip install --upgrade Gmsh 

import numpy as np
import matplotlib.pyplot as plt
import gmsh
import sys
import math
import Geo2Gmsh

gmsh.initialize()
gmsh.model.add("Colombia_Case")

exag = 1
Geo2Gmsh.counter = 0
Geo2Gmsh.counter1 = 0

Geo2Gmsh.create_surface(
    surf_id=1,
    file_name="topo.txt",
    samx=150,
    samy=115,
    v_ex=exag,
    show_color=False
)

Geo2Gmsh.create_surface(
    surf_id=2,
    file_name="sed_bottom.txt",
    samx=150,
    samy=115,
    v_ex=exag,
    show_color=False
)

Geo2Gmsh.create_surface(
    surf_id=3,
    file_name="moho.txt",
    samx=150,
    samy=115,
    v_ex=exag,
    show_color=False
)

volumes = Geo2Gmsh.volume_generation(
    num_loaded_surfaces = 3
    )

fault_1 = Geo2Gmsh.add_fault(
    file_name="fault_1.txt",
    v_ex=exag,
    surf_id = 2,
    fault_id = 1,
    dip = 90.,
    dip_dir = 45.,
    fault_len = 10000.,
)

fault_2 = Geo2Gmsh.add_fault(
    file_name="fault_2.txt",
    v_ex=exag,
    surf_id = 2,
    fault_id = 2,
    dip = 90.,
    dip_dir = 45.,
    fault_len = 10000.,
)

# wells = []
# for i in range(1, 12):
#     well = Geo2Gmsh.add_well(
#         surf_id=1,
#         well_id=i,
#         file_name=f"well_{i}.txt",
#         v_ex=exag
#     )
#     wells.append(well)

# Geo2Gmsh.local_refinement(
#     element_type = "well",
#     element_list = wells[0]+wells[1]+wells[2]+wells[3]+wells[4]+wells[5]+wells[6]+wells[7]+wells[8]+wells[9]+wells[10],
#     sampling =100,
#     Size_Min =500,
#     Size_Max =8000,
#     Dist_Min =5000,
#     Dist_Max =30000
# )

Geo2Gmsh.local_refinement(
    element_type = "fault",
    element_list = fault_1 + fault_2,
    sampling =800,
    Size_Min = 1000,
    Size_Max = 5000,
    Dist_Min = 10000,
    Dist_Max = 30000
)

# Geo2Gmsh.local_refinement(
#     element_type = "surface",
#     element_list = [2],
#     sampling =800,
#     Size_Min = 2000,
#     Size_Max = 5000,
#     Dist_Min = 10000,
#     Dist_Max = 40000
# )


#//////// block for field refinement ///////////
Geo2Gmsh.counter += 1
gmsh.model.mesh.field.add("Min", Geo2Gmsh.counter)
gmsh.model.mesh.field.setNumbers(Geo2Gmsh.counter, "FieldsList", [field for field in range(2, Geo2Gmsh.counter, 2)])
gmsh.model.mesh.field.setAsBackgroundMesh(Geo2Gmsh.counter)

#///////////////////////////////////////////////

gmsh.option.setNumber("Mesh.MeshSizeMax", 15000 )
gmsh.option.setNumber("Mesh.Algorithm3D", 2)  # cambiar el algoritmo puede ayudar
gmsh.option.setNumber("Mesh.MeshOnlyVisible", 1)


# Ajustes de tol erancia antes del mallado:
gmsh.option.setNumber("Geometry.Tolerance", 1)  # Por ejemplo, 1 metro
gmsh.option.setNumber("Mesh.ToleranceEdgeLength", 1)  # Opcional para el mallador

#Physical entities

# Geo2Gmsh.physical_group(
#     element_type = "well",
#     element_list = wells[0]+wells[1]+wells[2]+wells[3]+wells[4]+wells[5]+wells[6]+wells[7]+wells[8]+wells[9]+wells[10]
# )

Geo2Gmsh.physical_group(
    element_type = "fault",
    element_list = fault_1 + fault_2
)

Geo2Gmsh.physical_group(
    element_type = "surface",
    element_list = [1]
)

Geo2Gmsh.physical_group(
    element_type = "surface",
    element_list = [3]
)

Geo2Gmsh.physical_group(
    element_type = "volume",
    element_list = volumes
)
gmsh.model.geo.synchronize() #Super important. Not delete this line!!
gmsh.model.mesh.generate(3)

#///////// visulization (optional)
gmsh.option.setNumber("Mesh.SurfaceEdges", 1)
#gmsh.option.setNumber("Mesh.VolumeEdges", 1)
gmsh.option.setNumber("Mesh.SurfaceFaces", 1)
#gmsh.option.setNumber("Mesh.VolumeFaces", 1)  

#///////// output format ////////////
gmsh.write("outputs/Colombia_ex.msh")
gmsh.write("outputs/Colombia_ex.vtk")


#///// if working with sfepy the original vtk must be modified////
with open("outputs/Colombia_ex.vtk", "r") as f:
    lines = f.readlines()

with open("outputs/Colombia_ex.vtk", "w") as f:
    for line in lines:
        if "SCALARS CellEntityIds" in line:
            line = "SCALARS mat_id long\n"
        f.write(line)

gmsh.fltk.run()
gmsh.clear() 
gmsh.finalize()

Superficie 1 successfully created.
Superficie 2 successfully created.
Superficie 3 successfully created.
Volumes 1 succesfully created.
Volumes 2 succesfully created.
Fault 1 succesfully created.
Fault 2 succesfully created.
Mesh succesfully refined around faults.
physical_id_(faults): 1
physical_id_(surfaces): 2
physical_id_(surfaces): 3
physical_id_(volume 1): 4
physical_id_(volume 2): 5


In [11]:
#Se debe agarrar el archivo vtk de la carpeta output y pegarla en la carpeta donde se guarda el script, es decir simplemente fuera de la carpeta autput.
#Se corre este script para cambiar las lineas abajo escritas, lo que hace que sfepy reconozca las physical entities



In [ ]:
}}}}}